In [19]:
import os
import sys
import re
# Get the absolute path to the parent directory (assumes this file is in 'condensation')
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)
# autoreload modules
%load_ext autoreload
%autoreload 2
from typing import List, Dict, Tuple
from dataclasses import dataclass
import pandas as pd
from chatbot_api.providers.openai import OpenAIProvider
from chatbot_api.base import Role, Message

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Ponderings
  (1) hello ponderings
  (2) i think it makes sense to make the analysis optional, so that means we need to make the synthesize_batch function return args depending on a flag
      - this can be achieved with making the input to the synthesis be Argument class entities
  (3) the synthesizer class saves all inputs and outputs! but this should be more granular
      - input can be of three different types (init, sync, analysis). ofc we can just use the modulo of the indices to find which one it is, but for future reference i think should be divided with some more intuitive way of differentation. maybe just a ResponseType class with a List of Tuples: 
      - input_history -> List[((input_type), (str))]
          - str = LLM input string, 3 types
          - now input_history -> List[str] (where the strings are prompts)
      - output_history -> List[(output_type), (str)] 
          - str = LLM response string, 3 types

To Do
  (1) extend the parser
      - parse which comment indices the the LLM has given, in the analysis output, as sources for the changes 
      - then use them to show the admin some comments to see if the changes _actually_ make sense 
  (2) create a smarter flagging system
      - currently, we flag using relevance, which measures correlation between old Arguments and new comments
      - this is obviously stupid. new comments can have very different opinions than old Args. this ought not be punished.
      - it's better to have the LLM do some sort of qualitative analysis on whether the new Arg (or Args, plural) is _better_ or not
          - what is _better_, then? 
          - probably something like "do these new Args incorporate the ideas from the comments without losing their original information?"
          - maybe just: yes / no for simplicity
            - the "threshold" can be implicit, it's superfluous to have a relevance score that is just if-elsed out anyway 
  (3) keep track of some stats / metadata
      - stats for how often flagging happens and why
      - stats for 

In [20]:
@dataclass
class Argument:
    """
    A simplified argument class that tracks the main point and its history.
    The history allows us to see how the argument evolved over iterations.
    """
    main_argument: str
    argument_history: List[str]      # Tracks how this argument has changed over time
    latest_relevance_score: int    # The relevance score given by the model in the latest iteration
    latest_sources: List[str]        # Tracks the comments that led to the latest update
    latest_change: str
    latest_change_explained: str

    # currently not in use
    def update(self, new_arg: str):
        """Record a new version of this argument."""
        self.argument_history.append(self.main_argument)
        self.main_argument = new_arg

In [21]:
class ArgumentFlaggingSystem:
    def __init__(self, relevance_threshold: int = 8):
        """
        Initialize the argument flagging system.
        
        Args:
            relevance_threshold: Arguments with relevance scores below this value will be flagged
        """
        self.relevance_threshold = relevance_threshold

    def flag_low_relevance_arguments(self, 
                                   argument_analysis: Dict[str, List[Dict[str, str]]], 
                                   new_argument_texts: List[str]) -> List[Tuple[str, Dict[str, str]]]:
        """
        Identify arguments with low relevance scores that need admin review.
        
        Args:
            argument_analysis: List of argument analysis dictionaries from batch analysis, 
                             contains both single argument analysis and an entire batch summary
            new_argument_texts: List of new argument main points as strings
            
        Returns:
            List of tuples containing (main_point, metadata) for flagged arguments. Main point is from the initial synthesis output, not the analysis output. 
        """
        flagged_arguments = []
        
        # Extract the arguments analysis from the dictionary
        argument_analyses = argument_analysis.get('arguments', [])
        print(f'\nArguments used for flagging:')
        for argument in argument_analyses:
            print(f"  Argumentti {argument.get('index')}: {argument.get('argumentaatio')}")

        # TO DO: if the analysis happens to have more arguments (b/c of LLM wonkiness), what do we do? 
        # Currently, the zip function takes care of this (it's bijective), but is it optimal? at least a warning could be beneficial
        # Would also make sense to have some fall back for this... anyway, it's not a huge problem b/c high LLM accuracy
        if len(new_argument_texts) != len(argument_analyses):
            pass

        # Process each new argument and its analysis (PRESUMES THAT THE ORDER OF ARGUMENTS IS THE SAME IN BOTH LISTS, add a check for this)
        for input_arg, arg_analysis in zip(new_argument_texts, argument_analyses):
            should_flag = False
            flag_reason = []
            analysis_arg_string = arg_analysis.get('argumentaatio', '') # this is the LLM's latest version of the argument, given in the analysis output

            # Check if the argument text has changed
            if analysis_arg_string == input_arg:
                print(f"\n Flagger: an analysis argument matched the input argument")
                print(f"So, this string stayed the same: {analysis_arg_string}")
                # Text matches a saved argument, check relevance score
                try:
                    relevance_score = float(arg_analysis.get('relevanttius', '0'))
                    print(f"\nComparing relevance score of {relevance_score} to threshold {self.relevance_threshold}")
                    if relevance_score < self.relevance_threshold:
                        should_flag = True
                        flag_reason.append(f"Low relevance score: {relevance_score}")
                except (ValueError, TypeError) as e:
                    should_flag = True
                    flag_reason.append(f"\nInvalid relevance score format: {e}")
            else:
                print(f"\n Flagger: an analysis argument didn't match the input argument")
                print(f"\nNamely, OUTPUT != INPUT: ")
                print(f"{analysis_arg_string}")
                print("!=")
                print(f"{input_arg}")
                # New or modified argument text
                should_flag = True
                flag_reason.append("New argument is different from the analysis output argument")

            if should_flag:
                # Create metadata dictionary for the flagged argument
                metadata = {
                    "flag_reason": flag_reason,
                    "post_analysis_arg": analysis_arg_string,
                    "relevanttius": arg_analysis.get('relevanttius', 'Not available'),
                    "jaljitettavyys": arg_analysis.get('jaljitettavyys', 'Not available'),
                    "muutokset": arg_analysis.get('muutokset', 'Not available'),
                    "perustelut": arg_analysis.get('perustelut', 'Not available')
                }
                flagged_arguments.append((input_arg, metadata))
        
        return flagged_arguments

    async def get_confirmation_from_admin(self, 
                                        flagged_arguments: List[Tuple[str, Dict[str, str]]]) -> Dict[str, List[Argument]]:
        """
        Present flagged arguments to admin and get confirmation for each.
        
        Args:
            flagged_arguments: List of (main_point, metadata) tuples that need review
            
        Returns:
            List of approved Argument objects
        """
        approved_arguments = []
        
        if not flagged_arguments:
            print(f'\nFlagged arguments: {flagged_arguments}')
            return {"approved": approved_arguments, "rejected": []}
            
        print("\n=== Arguments Flagged for Review ===\n")
        
        for main_point, metadata in flagged_arguments:
            flag_reason = metadata.get('flag_reason', 'Unknown')
            if flag_reason == "New argument is different from the analysis output argument":
                print("-" * 50)
                print(f"Reviewing Argument: {main_point}")
                print(f"Reason: {main_point} doesn't match {metadata.get('post_analysis_arg', 'Error: Argument Not available')}")

            else:
                print("-" * 50)
                print(f"Reviewing Argument: {main_point}")
                print(f"Flag Reason: {flag_reason}")
                print("\nAnalysis:")
                print(f"- Relevance Score: {metadata.get('relevanttius', 'Not available')}")
                print(f"- Traceability: {metadata.get('jaljitettavyys', 'Not available')}")
                print(f"- Changes: {metadata.get('muutokset', 'Not available')}")
                print(f"- Reasoning: {metadata.get('perustelut', 'Not available')}")
                
            while True:
                response = input("\nAccept this argument for next batch? (y/n): ").lower()
                if response in ['y', 'n']:
                    break
                print("Please enter 'y' for yes or 'n' for no.")
            
            if response == 'y':
                # Parse the sources from traceability text
                sources = self._parse_sources(metadata.get('jaljitettavyys', ''))
                
                # Create new Argument object with the approved changes
                approved_argument = Argument(
                    main_argument=main_point,
                    argument_history=[],  # this needs to be populated with the corresponding argument history 
                    # it seems that using indices are reliable for history, so just give the Synthesizer.argument_history to the flagger
                    # then it can populate this initialization accordingly and discard the old version of the argument (or, maybe you could just update the same Argument?)
                    latest_relevance_score=float(metadata.get('relevanttius', '0')),
                    latest_sources=sources,
                    latest_change=metadata.get('muutokset', ''),
                    latest_change_explained=metadata.get('perustelut', '')
                )
                approved_arguments.append(approved_argument)
                print("Argument approved.")
            else:
                print("Argument rejected.")
        
        print("\n=== Review Complete ===")
        return {"approved": approved_arguments, "rejected": flagged_arguments}

    def _parse_sources(self, traceability_text: str) -> List[str]:
        """
        Parse the traceability text to extract source comments.
        
        Args:
            traceability_text: The text from the Jäljitettävyys field
            
        Returns:
            List of source comments
        """
        # Split the text by common separators to extract sources
        separators = [',', ';', '\n']
        for sep in separators:
            if sep in traceability_text:
                return [source.strip() for source in traceability_text.split(sep)] # not yet fully error-proof but good enough for now
            
    async def process_arguments(self, 
                              argument_analysis: Dict[str, List[Dict[str, str]]], 
                              new_argument_texts: List[str]) -> List[Argument]:
        """
        Main method to process arguments through flagging and admin review.
        
        Args:
            argument_analysis: Dictionary containing argument analyses and batch summary
            saved_arguments: List of previously saved Argument objects
            new_argument_texts: List of new argument main points
            
        Returns:
            List of approved Argument objects
        """
        # Flag arguments that need review
        flagged_args = self.flag_low_relevance_arguments(
            argument_analysis, 
            new_argument_texts
        )
        
        # Get admin confirmation and return approved Argument objects
        return await self.get_confirmation_from_admin(flagged_args)

In [22]:
# BECAUSE THE INIT IS EMPTY, WE CAN MAKE THIS INTO AN OBJECT
class OutputParser:
    def __init__(self):
        pass 

    # -----------------------
    # INITIAL BATCH parsing
    # -----------------------

    def parse_arguments_from_init(self, response: str) -> List[str]:
        """Extract argument main points from LLM response."""
        pattern = r'<ARGUMENTS>(.*?)</ARGUMENTS>'
        match = re.search(pattern, response, re.DOTALL)
        if not match:
            return []

        arguments = []
        for line in match.group(1).strip().split('\n'):
            line = line.strip()
            if line.startswith('ARGUMENTTI '): # Check for index? And maybe use latest_index + 1, if no index
                colon_index = line.find(':')
                if colon_index != -1:
                    arguments.append(line[colon_index + 1:].strip())
        
        return arguments

    # -----------------------
    # SYNTHESIS parsing
    # -----------------------

    def parse_arguments_from_synthesis(self, response: str):
        return self.parse_arguments_from_init(response) # currently identical implementation
   
    # -----------------------
    # ANALYSIS parsing
    # -----------------------
    
    def parse_arguments_from_analysis(self, analysis_text) -> List[Dict[str, str]]:
        """
        Parses the individual arguments' analysis section.
        Returns a list of dictionaries where each dictionary represents a single argument.
        """
        argument_pattern = re.compile(
            r"ARGUMENTTI (\d+):\s*"
            r"Argumentaatio:\s*(.*?)\s*(?=Jäljitettävyys:)"
            r"Jäljitettävyys:\s*(.*?)\s*(?=Muutokset:)"
            r"Muutokset:\s*(.*?)\s*(?=Relevanttius:)"
            r"Relevanttius:\s*(.*?)\s*(?=Perustelut:)"
            r"Perustelut:\s*(.*?)\s*(?=ARGUMENTTI|\s*</ANALYYSI>)",
            re.DOTALL
        )
        # Remove extra whitespace and normalize newlines
        cleaned_text = re.sub(r'\s+', ' ', analysis_text).strip()
            
        matches = argument_pattern.finditer(cleaned_text)
        arguments = []

        for match in matches:
            arguments.append({
                "index": match.group(1),
                "argumentaatio": match.group(2).strip(),
                "jaljitettavyys": match.group(3).strip(),
                "muutokset": match.group(4).strip(),
                "relevanttius": match.group(5).strip(),
                "perustelut": match.group(6).strip(),
            })
        return arguments

    def parse_summary_from_analysis(self, analysis_text) -> Dict[str, str]:
        """
        Parses the summary section of the output.
        Returns a dictionary containing the summary information.
        """
        summary_pattern = re.compile(
            r"<YHTEENVETO>\s*"
            r"Yhteenveto:\s*\[(.*?)\]\s*"
            r"Perustelut:\s*\[(.*?)\]\s*"
            r"Kehityskohteet:\s*\[(.*?)\]\s*</YHTEENVETO>",
            re.DOTALL
        )
        match = summary_pattern.search(analysis_text)
        
        if match:
            return {
                "yhteenveto": match.group(1).strip(),
                "perustelut": match.group(2).strip(),
                "kehityskohteet": match.group(3).strip(),
            }
        return {}

    def parse_analysis(self, analysis_text) -> Dict[str, List[Dict[str, str]]]:
        """
        Combines the parsed arguments and summary into a single structure.
        """
        return {
            "arguments": self.parse_arguments_from_analysis(analysis_text),
            "summary": self.parse_summary_from_analysis(analysis_text), # although parse_summary returns Dict[str, str], not List[Dict[str, str]]
        }

In [23]:
class ArgumentSynthesizer:
    def __init__(self, llm_provider, batch_size, flagger, parser):
        self.parser = parser # would be nice to have types for parsers, flaggers and providers in this code... (now Any)
        self.flagger = flagger
        self.llm_provider = llm_provider
        self.batch_size = batch_size
        self.current_arguments: List[Argument] = []
        self.output_history = []
        self.input_history = []
        self.n_iterations = 0
        
        # Template for initial argument creation, now in Finnish with more flexible argument count
        self.INITIAL_TEMPLATE = """
            ### Ohjeet:
            1. Käy läpi seuraavat kommentit aiheesta: "{topic}"
            2. Muodosta 2-5 pääargumenttia, jotka kuvaavat keskeisiä näkökulmia aiheeseen:
            - Pidä argumentit tiiviinä, jotta ne edustavat selkeästi erottuvaa näkökantaa
            - Argumenttien tulisi yhdessä kattaa kaikki kommenteissa mainitut ajatukset
            - Jos kommenteissa esiintyy uusi näkökulma, suosi uuden argumentin luomista vanhojen muokkaamisen sijaan
            - Varmista, että argumentit on kirjoitettu selkeästi ja kattavasti
            3. Huomioi erityisesti:
            - Argumenttien määrä riippuu kommenttien sisällön monipuolisuudesta
            - On parempi luoda useita tiiviitä argumentteja kuin pieni määrä liian pitkiä argumentteja

            ### Analysoitavat kommentit:
            {comments_text}

            ### Tulostusmuoto:
            <ARGUMENTS>
            ARGUMENTTI 1: [Ensimmäinen kattava argumentti]
            ARGUMENTTI 2: [Toinen kattava argumentti]
            ARGUMENTTI 2: [Kolmas kattava argumentti]
            </ARGUMENTS>
        """
        
        # Template for synthesis, also in Finnish
        self.SYNTHESIS_TEMPLATE = """
            ### Ohjeet:
            1. Käy läpi nykyiset argumentit ja uudet kommentit aiheesta: "{topic}"
            2. Kehitä ja täydennä argumenttirakennetta:
            - Pidä argumentit tiiviinä, jotta ne edustavat selkeästi erottuvaa näkökantaa
            - Jaa argumentit, jos ne sisältävät selvästi erillisiä näkökantoja
            - Luo uusia argumentteja, jos kommentit paljastavat käsittelemättömiä näkökulmia
            - Mikäli uusi näkökanta liittyy johonkin aiempaan argumenttiin vahvasti, muokkaa olemassa olevia argumentteja kuvaamaan paremmin keskustelun koko laajuutta
            3. Tärkeää:
            - Argumenttien lopullinen määrä määräytyy sisällön perusteella
            - Päätös vanhan argumentin muokkaamisen ja uuden argumentin luomisen välillä riippuu siitä, kuinka helposti uuden näkökulman voi sitoa vanhaan argumenttiin tekemättä argumentista liian pitkää
            - Jokaisen argumentin tulee olla selkeästi erillinen muista argumenteista

            ### Nykyiset argumentit:
            {existing_arguments}

            ### Uudet kommentit:
            {new_comments}

            ### Tulostusmuoto:
            <ARGUMENTS>
            ARGUMENTTI 1: [Päivitetty argumentti]
            ARGUMENTTI 2: [Päivitetty argumentti]
            ARGUMENTTI 3: [Sama argumentti kuin aiemmin]
            ARGUMENTTI 4: [Päivitetty argumentti]
            ARGUMENTTI 5: [Sama argumentti kuin aiemmin]
            ARGUMENTTI 6: [Täysin uusi argumentti]
            </ARGUMENTS>
        """

        self.BATCH_VALIDATION_TEMPLATE = """
            ### Ohjeet:
            1. Käy läpi seuraavat kommentit ja vanhat sekä uudet argumentit aiheesta: "{topic}".
            2. Arvioi uudet argumentit yksitellen seuraavien kohtien mukaan:
            - Jäljitettävyys: Kirjaa, mitkä kommentit vaikuttivat kunkin muuttuneen argumentin muutoksiin. Tai jos uskot argumentin olevan täysin uusi, kirjaa se.
            - Muutokset: Ilmoita, mitä lisäyksiä, poistamisia tai muutoksia argumentissa on tehty. Jos argumentti on täysin uusi, voit jättää tämän kohdan tyhjäksi.
            - Relevanttius: Anna jokaiselle argumentille pisteytys asteikolla 1-10 sen mukaan, kuinka hyvin muutokset vastaavat käytettyjä kommentteja. Tai jos argumentti on täysin uusi, arvioi sen relevanssi kommentteihin.
            3. Luo yleinen tiivistelmä: Luo lyhyt, korkeatasoinen yhteenveto koko analysoidusta erästä. Mitä pääkohtia huomioit kommenteissa, ja miten argumentit ovat kehittyneet niiden mukaan?
            4. Ehdota kehityskohteita. Muotoile parannusehdotukset siten, että ne ovat ohje seuraavaa chatbotin suorittamaa analyysia varten.
            
            ### Analysoitavat tiedot:
            Kommentit:
            {comments_text}

            Vanhat argumentit:
            {old_arguments_text}

            Uudet argumentit:
            {new_arguments_text}

            ### Tulostusmuoto:
            <ANALYYSI>
            ARGUMENTTI 1:
            Argumentaatio: [Argumentin pääsisältö]
            Jäljitettävyys: [Kommentit, jotka vaikuttivat muutoksiin (jätä tyhjäksi, jos uusi argumentti)]
            Muutokset: [Mitkä asiat muuttuivat]
            Relevanttius: [Relevanssipisteytys 1-10]
            Perustelut: [Perustelut annetuille pisteille]
            ARGUMENTTI 2:
            Argumentaatio: [Argumentin pääsisältö]
            Jäljitettävyys: [Kommentit, jotka vaikuttivat muutoksiin (jätä tyhjäksi, jos uusi argumentti)]
            Muutokset: [Mitkä asiat muuttuivat]
            Relevanttius: [Relevanssipisteytys 1-10]
            Perustelut: [Perustelut annetuille pisteille]
            </ANALYYSI>

            <YHTEENVETO>
            Yhteenveto: [Yleinen yhteenveto argumenttien muutoksista, poistoista ja lisäyksistä]
            Perustelut: [Millä tavalla eri muutokset on perusteltu]
            Kehityskohteet: [Parannusehdotukset]
            </YHTEENVETO>
        """

    async def process_comments(self, comments: List[str], topic: str, use_analysis: bool) -> List[Argument]:
        """Process all comments in batches, focusing on argument evolution."""
        
        # Process initial batch to create 2-4 arguments
        if not self.current_arguments:
            initial_batch = comments[:self.batch_size]
            await self._process_initial_batch(initial_batch, topic)
        
        # Process remaining batches
        for i in range(self.batch_size, len(comments), self.batch_size):
            batch = comments[i:i + self.batch_size]
            await self._synthesize_batch(batch, topic, use_analysis)
            
        return self.current_arguments

    async def _process_initial_batch(self, comments: List[str], topic: str):
        """Generate initial set of 2-4 arguments from first batch of comments."""
        comments_text = "\n".join(f"Kommentti {i+1}: {comment}" 
                                for i, comment in enumerate(comments))
        
        prompt = self.INITIAL_TEMPLATE.format(
            topic=topic,
            comments_text=comments_text
        )
        
        response = await self.llm_provider.generate([Message(Role.USER, prompt)])
        new_arguments = self.parser.parse_arguments_from_init(response.content)
        
        # Save input and output for future analysis
        self.output_history.append(response.content)
        self.input_history.append(prompt)

        # Create initial arguments
        self.current_arguments = [
            Argument(
                main_argument=arg,
                argument_history=[], 
                latest_relevance_score=0, # placeholder for initialization
                latest_sources=[],
                latest_change="Initial creation",
                latest_change_explained="Initial creation"
            ) for arg in new_arguments
        ]

        self.n_iterations += 1

    async def _synthesize_batch(self, new_comments: List[str], topic: str, use_analysis: bool) -> List[Argument]:
        """Synthesize new comments with existing arguments."""
        current_args_text = "\n".join(
            f"Argumentti {i+1}: {arg.main_argument}" 
            for i, arg in enumerate(self.current_arguments)
        )
        
        new_comments_text = "\n".join(
            f"Kommentti {i+1}: {comment}" 
            for i, comment in enumerate(new_comments)
        )
        
        prompt = self.SYNTHESIS_TEMPLATE.format(
            topic=topic,
            existing_arguments=current_args_text,
            new_comments=new_comments_text
        )
        
        response = await self.llm_provider.generate([Message(Role.USER, prompt)])
        new_argument_points = self.parser.parse_arguments_from_synthesis(response.content)

        # Save input and output for future analysis
        self.output_history.append(response.content)
        self.input_history.append(prompt)
        
        # TO DO: make 
        if use_analysis:
            await self.analyze_arguments(new_argument_points, new_comments, topic) # maybe break this into two parts: 

        self.n_iterations += 1
        return [] # placeholder for now, but should return 

    async def analyze_arguments(self, new_argument_points: List[str], new_comments: List[str], topic) -> List[str]:
        """Use an LLM to analyze the arguments and provide feedback."""
        # populate the template with the arguments and comments
        old_arguments_text = "\n".join(
            f"Vanha argumentti {i+1}: {arg}" 
            for i, arg in enumerate(self.current_arguments)
        )

        comments_text = "\n".join(f"Kommentti {i+1}: {comment}" 
            for i, comment in enumerate(new_comments))

        new_arguments_text = "\n".join(
            f"Uusi argumentti {i+1}: {arg}" 
            for i, arg in enumerate(new_argument_points)
        )

        prompt = self.BATCH_VALIDATION_TEMPLATE.format(
            topic=topic,
            comments_text=comments_text,
            old_arguments_text=old_arguments_text,
            new_arguments_text=new_arguments_text
        )

        response = await self.llm_provider.generate([Message(Role.USER, prompt)])
        
        # Save input and output for future analysis
        self.output_history.append(response.content)
        self.input_history.append(prompt)

        # Parse the analysis text to get the arguments and summary
        parsed_data = self.parser.parse_analysis(response.content) 

        # Flag arguments with low relevance for removal. Wait for user confirmation.
        flagged_arguments = await self.flagger.process_arguments(argument_analysis=parsed_data, 
                                                      new_argument_texts=new_argument_points) # run this to see whether the flagging works
        
        # return the list of accepted strings (argument strings)


    # Something to ponder for implementation later
    async def get_argument_evolution_report(self) -> str: 
        """Generate a readable report of how arguments evolved throughout."""
        pass

In [24]:
# config
api_key = os.getenv("OPENAI_API_KEY")
model = "gpt-4o-mini-2024-07-18" # fast and cheap
# model="gpt-4o-2024-11-20"      # slow, expensive, but powerful
openai_provider = OpenAIProvider(api_key, model)
data_source_path = os.path.join(parent_dir, 'data', 'sources', 'kuntavaalit2021.csv')
output_path = os.path.join(parent_dir, 'condensation', 'results', 'current_results', 'v0_results.txt')
n_comments = 20
batch_size = 10
relevance_threshold = 8
topic = "?"

# NOT IN USE YET BUT USEFUL
# Choose a subset of comments to process
"""question_index = 10
explanation_column_name = f'q{question_index}.explanation_fi'
likert_column_name = f'q{question_index}.answer' """

# get comments
df = pd.read_csv(data_source_path)
comment_indices = df['q9.explanation_fi'].dropna()[:n_comments].index.tolist()
comments = df.loc[comment_indices, 'q9.explanation_fi'].tolist()

# process arguments
parser = OutputParser()
flagger = ArgumentFlaggingSystem(relevance_threshold)
processor = ArgumentSynthesizer(openai_provider, batch_size, flagger, parser)
argument_analysis = await processor.process_comments(comments,topic, True)

# print results
lol = """ for arg in arguments:
    print(arg.main_argument)
    print(arg.argument_history)
    print() """


Arguments used for flagging:
  Argumentti 1: Pääomaverotuksen tulisi olla osittain kunnallista, mikä mahdollistaisi paikallisten tarpeiden huomioimisen. Veroprosentin tulisi kuitenkin olla matalampi ja portaaton, jotta se ei kohtele epäoikeudenmukaisesti esimerkiksi metsänomistajia, jotka voivat joutua maksamaan suuren osan tulostaan yhdessä vuodessa. Lisäksi on tärkeää, että veromalli ei rajoita työntekoa ja yrittämistä, sillä verotus on keskeinen tekijä vetovoimassa ja paikallisten palveluiden käytössä.
  Argumentti 2: Suomi on jo tunnettu korkeasta progressiivisesta verotuksestaan, ja verotuksen lisääminen ei välttämättä tuo merkittäviä etuja kunnalliseen verokertymään, sillä eniten ansaitsevien osuus on pieni. Tämän vuoksi olisi tärkeämpää keskittyä kannustamaan kulutusta ja paikallisten palveluiden käyttöä. Verotuksen kiristäminen nykyisestä ei edistä kansalaisten ostovoimaa, joka on taloudellisen kasvun ja korkean työllisyyden edellytys.
  Argumentti 3: Progressiivinen verotus o